<a href="https://colab.research.google.com/github/stefanziog/Quantized-Deep-Neural-Networks/blob/main/aware/mnist_aware.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
! pip uninstall -y tensorflow
! pip install -q tf-nightly
! pip install -q tensorflow-model-optimization

Found existing installation: tensorflow 2.6.0
Uninstalling tensorflow-2.6.0:
  Successfully uninstalled tensorflow-2.6.0
     |████████████████████████████████| 483.4 MB 31 kB/s 
     |████████████████████████████████| 1.3 MB 42.4 MB/s 
     |████████████████████████████████| 463 kB 49.1 MB/s 
     |████████████████████████████████| 12.8 MB 144 kB/s 
     |████████████████████████████████| 5.8 MB 22.6 MB/s 
     |████████████████████████████████| 2.3 MB 49.3 MB/s 
     |████████████████████████████████| 211 kB 8.5 MB/s 


In [ ]:
import tempfile
import os

import tensorflow as tf

from tensorflow import keras

In [ ]:
# load train and test dataset
def load_dataset():
# load dataset
  mnist = keras.datasets.mnist
  (train_images, train_labels), (test_images, test_labels) = mnist.load_data()
	
  return train_images, train_labels, test_images, test_labels

# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

In [ ]:
# load dataset
train_images, train_labels, test_images, test_labels = load_dataset()

# prepare pixel data
train_images, test_images = prep_pixels(train_images, test_images)

#load model
model = tf.keras.models.load_model('/content/gdrive/MyDrive/Διπλωματική Κώδικες τελικοί/saved_models/mnist.h5')

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
#Define Quantization Model
import tensorflow_model_optimization as tfmot

quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

q_aware_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_layer (QuantizeLay  (None, 28, 28)            3         
er)                                                              
_________________________________________________________________
quant_reshape (QuantizeWrap  (None, 28, 28, 1)         1         
per)                                                             
_________________________________________________________________
quant_conv2d (QuantizeWrapp  (None, 14, 14, 32)        1635      
er)                                                              
_________________________________________________________________
quant_max_pooling2d (Quanti  (None, 5, 7, 32)          1         
zeWrapper)                                                       
_________________________________________________________________
quant_conv2d_1 (QuantizeWra  (None, 3, 4, 64)          1

In [ ]:
#train and evaluate model against baseline
train_images_subset = train_images[0:1000]  # out of 60000
train_labels_subset = train_labels[0:1000]

q_aware_model.fit(train_images_subset, train_labels_subset,
                  batch_size=500, epochs=5, validation_split=0.1)

Epoch 1/5
2/2 [==============================] - 2s 512ms/step - loss: 0.1330 - accuracy: 0.9844 - val_loss: 0.0653 - val_accuracy: 1.0000
Epoch 2/5
2/2 [==============================] - 0s 204ms/step - loss: 0.0752 - accuracy: 0.9900 - val_loss: 0.0606 - val_accuracy: 1.0000
Epoch 3/5
2/2 [==============================] - 0s 203ms/step - loss: 0.0569 - accuracy: 0.9900 - val_loss: 0.0522 - val_accuracy: 1.0000
Epoch 4/5
2/2 [==============================] - 0s 210ms/step - loss: 0.0420 - accuracy: 0.9956 - val_loss: 0.0400 - val_accuracy: 1.0000
Epoch 5/5
2/2 [==============================] - 0s 201ms/step - loss: 0.0296 - accuracy: 0.9989 - val_loss: 0.0372 - val_accuracy: 1.0000


In [ ]:
#Create Quantize model for TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmphhinnz8f/assets


INFO:tensorflow:Assets written to: /tmp/tmphhinnz8f/assets


In [ ]:
#See persistance of accuracy from tf to tflite
import numpy as np

def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

In [ ]:
#Evaluate the quantized model
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

_, q_aware_model_accuracy = q_aware_model.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Quant test accuracy:', q_aware_model_accuracy)

Baseline test accuracy: 0.9865999817848206
Quant test accuracy: 0.9879999756813049


In [ ]:
#Check the size of the models
# Create float TFLite model.
float_converter = tf.lite.TFLiteConverter.from_keras_model(model)
float_tflite_model = float_converter.convert()

# Measure sizes of models.
_, float_file = tempfile.mkstemp('.tflite')
_, quant_file = tempfile.mkstemp('.tflite')

with open(quant_file, 'wb') as f:
  f.write(quantized_tflite_model)

with open(float_file, 'wb') as f:
  f.write(float_tflite_model)

print("Float model in Mb:", os.path.getsize(float_file) / float(2**20))
print("Quantized model in Mb:", os.path.getsize(quant_file) / float(2**20))

INFO:tensorflow:Assets written to: /tmp/tmpv6i_rnzf/assets


INFO:tensorflow:Assets written to: /tmp/tmpv6i_rnzf/assets


Float model in Mb: 0.5430374145507812
Quantized model in Mb: 0.14563751220703125
